# Demo - Support Vector Machines
### Use the <a href='https://mybinder.org/v2/gh/ihler/ml-demos/HEAD?filepath=notebooks%2FDemo-Live-SVM.ipynb'>Live Binder demo</a>

In [2]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.set_cmap('jet');
%matplotlib inline

import mltools as ml
import mltools.datagen

import ipywidgets as widgets       # use for interactive plots with "inline" plotting
from ipyevents import Event 
from IPython.display import display

class Struct(): pass

#### Interactive demo
* Click to place initial parameters
* Change kernel parameters (Gaussian bandwidth, polynomial degree)
* Change optimization parameters (max iterations, soft margin slack penalty R)

In [5]:
import mltools.kernel as kernel

opts = Struct(); opts.T = 10; opts.ktype='Gaussian'; opts.P=3; opts.sig=.5; opts.R = 100.

svmG = kernel.kernelClassify2(); svmG.kernel = lambda a,b: kernel.gaussian(a,b,opts.sig)

T = widgets.IntSlider(opts.T,description='T');
sig = widgets.FloatLogSlider(value=.5, min=-3,max=1,step=.01, description='alpha')
ktype = widgets.Dropdown(options=['Linear', 'Poly', 'Gaussian'],value='Gaussian',description='Kernel type:')
P = widgets.IntSlider(opts.P,min=1,max=9,description='Degree');
R = widgets.FloatLogSlider(value=opts.R, min=-3,max=3,step=.01, description='R')

def myPlot(X,Y):
    if (len(Y)>1 and np.var(Y)>0):
        Y[Y!=0] = 1
        svmG.train(X,Y, opts.R, maxIter=opts.T)
        ml.plotClassify2D(svmG,X,Y)
    elif len(X>0):
        ml.plotClassify2D(None,X,Y)
        
data = ml.datagen.MouseDataWidget(plot=myPlot)

def set_opts(T,ktype,sig,P,R):
    opts.T = T; opts.R = R; opts.sig = sig; 
    if ktype=='Gaussian': svmG.kernel = lambda a,b: kernel.gaussian(a,b,opts.sig)
    elif ktype=='Linear': svmG.kernel = lambda a,b: kernel.linear(a,b)
    elif ktype=='Poly':   svmG.kernel = lambda a,b: kernel.poly(a,b,P)
    data.update_plot()

instructions = """
&nbsp&nbsp Left click to add points, right click to remove nearest point.<br>
&nbsp&nbsp Use shift/alt/ctrl combinations to control class value when placing points.
"""
_ = widgets.interactive_output(set_opts, {'T': T, 'ktype': ktype, 'sig': sig, 'P':P, 'R':R})
controls = widgets.VBox([widgets.Label("Kernel:"),ktype,P,sig,
                         widgets.Label("Training:"),T,R]) 

grid = widgets.GridspecLayout(6, 5)
grid[:,0:2] = data.image
grid[0,2:] = widgets.HTML(instructions)
grid[1,2:] = widgets.Label("Controls:")
grid[2:,2:] = controls
display(grid)   

<Figure size 432x432 with 0 Axes>

GridspecLayout(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01\xb0\x08…